In [1]:
import easyocr
import os
import cv2
import pandas as pd
from tqdm import tqdm

In [3]:
reader = easyocr.Reader(['de'])
dataset_path = "/home/tguenther/Data/GERALD_v1_5_thomas/cropped_images_speed_sorted"

results = []

def categorize_resolution(width):
    if width < 50:
        return 'niedrig'
    elif width <= 100:
        return 'mittel'
    else:
        return 'hoch'

for folder_name in sorted(os.listdir(dataset_path)):
    true_label = int(folder_name.lstrip('0'))
    folder_path = os.path.join(dataset_path, folder_name)

    for img_name in tqdm(os.listdir(folder_path)):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        height, width = img.shape[:2]

        # OCR durchführen
        result = reader.readtext(img_path, detail=0)
        predicted_label = int(result[0]) if result and result[0].isdigit() else None
        correct = predicted_label == true_label

        # Ergebnisse speichern
        results.append({
            'path': img_path,
            'true_label': true_label,
            'predicted_label': predicted_label,
            'correct': correct,
            'width': width,
            'height': height,
            'resolution_category': categorize_resolution(width)
        })

# Ergebnisse als DataFrame speichern
df_results = pd.DataFrame(results)

100%|██████████| 23/23 [00:00<00:00, 375.65it/s]


In [4]:
df_results

,path,true_label,predicted_label,correct,width,height,resolution_category
0,/home/tguenther/Data/GERALD_v1_5_thomas/croppe...,2,NaN,False,10,15,niedrig
1,/home/tguenther/Data/GERALD_v1_5_thomas/croppe...,2,NaN,False,10,13,niedrig
2,/home/tguenther/Data/GERALD_v1_5_thomas/croppe...,2,NaN,False,12,13,niedrig
3,/home/tguenther/Data/GERALD_v1_5_thomas/croppe...,2,NaN,False,14,17,niedrig
4,/home/tguenther/Data/GERALD_v1_5_thomas/croppe...,2,NaN,False,34,35,niedrig
...,...,...,...,...,...,...,...
1100,/home/tguenther/Data/GERALD_v1_5_thomas/croppe...,16,NaN,False,96,71,mittel
1101,/home/tguenther/Data/GERALD_v1_5_thomas/croppe...,16,16.0,True,39,44,niedrig
1102,/home/tguenther/Data/GERALD_v1_5_thomas/croppe...,16,NaN,False,15,21,niedrig
1103,/home/tguenther/Data/GERALD_v1_5_thomas/croppe...,16,16.0,True,20,26,niedrig


In [ ]:
df_results['resolution_category'].value_counts()
'''
Kategorien zusammenfassen.
'''

resolution_category
niedrig    907
mittel     169
hoch        29
Name: count, dtype: int64

In [ ]:
# Accuracy pro Kategorie
accuracy_by_resolution = df_results.groupby('resolution_category')['correct'].mean()
print(accuracy_by_resolution)
'''
Extrem niedrige Accuracy (< 30%) deutet meist darauf hin, dass grundlegende Probleme existieren, sodass der Zusammenhang mit der Auflösung möglicherweise weniger entscheidend ist als andere Einflussgrößen.
'''

# # Metrische Korrelation (z.B. Breite vs Accuracy)
# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(10, 6))
# sns.boxplot(x='resolution_category', y='width', hue='correct', data=df_results)
# plt.title('OCR-Korrektheit in Abhängigkeit von der Bildbreite')
# plt.xlabel('Auflösungskategorie')
# plt.ylabel('Bildbreite (Pixel)')
# plt.show()
# '''
# 1. Es gibt eine leichte Tendenz, dass Bilder mit höherer Auflösung besser erkannt werden. Zu erkennen an den Medianen. Bei hoch (False) ist eine hohe Varianz zu erkennen, d.h. nicht nur die Auflösung bestimmt die Qualität der OCR, sondern eventuell auch Bildqualität oder andere Faktoren sind noch entscheidend.
# 2. Ich kann keinen Grenzwert erkennen ab dem die OCR Erkennung zuverlässig arbeitet.
# 3. Hoch: Trotz hoher Auflösung gibt es noch falsche OCR Erkennung.
# 4. Die statistische Aussagekraft ist bei hoch recht leicht gewichtig auf Grund der geringen Sampleanzahl. Runter setzen auf unter 90
# '''

resolution_category
hoch       0.310345
mittel     0.142012
niedrig    0.060639
Name: correct, dtype: float64


In [ ]:
# Ergebnisse in DataFrame umwandeln und zählen
df_counts = pd.Series(resolution_counts).value_counts()

# Ausgabe der Sample-Anzahl pro Kategorie
print("Anzahl der Samples pro Auflösungskategorie:")
print(df_counts)
